## Feature Development Extraction: Neurophysiology [dyskinesia project]




<b> Content </b>


<b> Aperiodic estimates </b>
Relevant literature:
- Periodic and a-periodic components relevance and interaction, different reasons (per + a-per) for signal changes observed within a specific bandwidth. Aperiodic component (complicated) vs exponent (1/f) (Donoghue, ..., Shestyuk & Voytek, Nature Neurosc 2020 : https://www.nature.com/articles/s41593-020-00744-x)
- cycle-by-cycle features: bycycle toolbox (Cole & Voytek, J of Neurophys 2019, https://journals.physiology.org/doi/full/10.1152/jn.00273.2019)
- aperiodic component, PD severity, and cortico-subcortico-activity, Bush & Zou, Richardson, bioRxiv 2023 https://www.biorxiv.org/content/10.1101/2023.02.08.527719v1?rss=1

<b> Periodic component analysis: </b> 
- Try Wavelet Dceomposition vs Welch (tapered) Spectral Decomposition





### 0. Loading packages and functions, defining paths



In [1]:
# Importing Python and external packages
import os
import sys
import importlib
import json
import csv
from dataclasses import dataclass, field, fields
from collections import namedtuple
from typing import Any
from itertools import compress
from pathlib import Path
import pandas as pd
import numpy as np
import sklearn as sk
from scipy.stats import pearsonr, mannwhitneyu
import statsmodels as sm

import matplotlib.pyplot as plt
from  matplotlib import __version__ as plt_version
from  scipy import __version__ as scipy_version
from scipy import signal, stats
# from array import array
# import datetime as dt
# #mne
# import mne_bids
# import mne


In [2]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
# print('mne_bids', mne_bids.__version__)
# print('mne', mne.__version__)
print('sci-py', scipy_version)
print('sci-kit learn', sk.__version__)
print('matplotlib', plt_version)
print('statsmodels', sm.__version__)
## DEC 11 2022:
# Python sys 3.9.0 (default, Nov 15 2020, 08:30:55)
# pandas 1.4.4
# numpy 1.23.3
# sci-py 1.9.3
# sci-kit learn 1.1.3
# matplotlib 3.5.3

## Updated ENV DEC 11 2022:
# Python sys 3.10.8  (main, Nov  4 2022, 13:42:51)
# pandas 2.1.4  -> downgraded to pandas 1.5.3 bcs of pickle functionality (pd.indexing error)
# numpy 1.26.2
# sci-py 1.11.4
# sci-kit learn 1.3.2
# matplotlib 3.8.2
# statsmodels 0.14.0

Python sys 3.9.0 (default, Nov 15 2020, 08:30:55) [MSC v.1916 64 bit (AMD64)]
pandas 1.4.4
numpy 1.23.3
sci-py 1.9.3
sci-kit learn 1.1.3
matplotlib 3.5.3
statsmodels 0.14.0


In [3]:
def get_project_path_in_notebook(
    subfolder: str = '',
):
    """
    Finds path of projectfolder from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()

    while path[-20:] != 'dyskinesia_neurophys':

        path = os.path.dirname(path)
    
    return path

In [4]:
# define local storage directories
projectpath = get_project_path_in_notebook()
codepath = os.path.join(projectpath, 'code')
figpath = os.path.join(projectpath, 'figures')
datapath = os.path.join(projectpath, 'data')

In [5]:
os.chdir(codepath)
# own utility functions
import utils.utils_fileManagement as utilsFiles
import utils.utils_windowing as utilsWindows
from utils.utils_fileManagement import (get_project_path,
                                        load_class_pickle,
                                        save_class_pickle,
                                        mergedData,
                                        correct_acc_class)
# own data preprocessing functions
import lfpecog_preproc.preproc_data_management as dataMng
import lfpecog_preproc.preproc_filters as fltrs
# own data exploration functions
import lfpecog_features.feats_read_proc_data as read_data
import lfpecog_features.feats_spectral_baseline as specBase
import lfpecog_features.feats_spectral_features as spectral
import lfpecog_features.feats_spectral_helpers as specHelp
import lfpecog_features.feats_helper_funcs as ftHelp

import lfpecog_preproc.preproc_import_scores_annotations as importClin
import lfpecog_analysis.import_ephys_results as importResults
# import lfpecog_analysis.get_acc_derivs as accDerivs


import lfpecog_plotting.plotHelpers as plotHelp

### 1. Load preprocessed and merged Sub-Data

#### load ACC pickles

In [ ]:
subs_to_plot = [
    '008', 
    # '009', '010', '012', 
    # '013', '014', '016'
]

data_version = 'v4.0'
mins_recording = []

for sub in subs_to_plot:
    # load Acc-detected movement labels
    acc = load_class_pickle(os.path.join(
        get_project_path('data'),
        'merged_sub_data', data_version, f'sub-{sub}',
        f'{sub}_mergedData_{data_version}_accLeft.P'
    ))
    # acc = correct_acc_class(acc)

    # mins_recording.append(acc.data.shape[0] / acc.fs / 60)

#### load ephys pickle

WINDOWED DATA

In [ ]:
subs_to_plot = [
    # '008', '009', '010', '012', 
    # '013', '014',
    '012',
]
task = 'rest'
data_version = 'v3.0'
mins_recording = []

# for sub in subs_to_plot:
#     # load Acc-detected movement labels
#     data = load_class_pickle(os.path.join(
#         get_project_path('data'), 'windowed_data_classes_60s',
#         data_version, f'sub-{sub}', task,
#         f'{sub}_mneEpochs_{task}_{data_version}_win60s_overlap0.5.P'
#     ))
#     # acc = correct_acc_class(acc)

# data.list_mne_objects[0].times.shape
# data.list_mne_objects[0].ch_names
# data.list_mne_objects[0].get_data().shape


MERGED DATA per DATATYPE (source)


In [ ]:
# LOAD TASK DATA BEFORE MERGING
sub = '019'
data_version = 'v4.0'
dType = 'ecog_right'

path = os.path.join(get_project_path('data'),
                    'preprocessed_data',
                    f'sub-{sub}', 'v4.0',)
f_data = f'data_{sub}_Rest_StimOffDopa35_{data_version}_{dType}_2048Hz.npy'
c_data = f'names_{sub}_Rest_StimOffDopa35_{data_version}_{dType}.csv'

d = np.load(os.path.join(path, f_data))
cols = pd.read_csv(os.path.join(path, c_data))

In [ ]:
# LOAD MERGED PICKLE

data_version = 'v4.0'
sub = '019'
dat = {}
dType = 'lfp_right'
dat[dType] = load_class_pickle(os.path.join(
        get_project_path('data'), 'merged_sub_data',
        'v4.0', f'sub-{sub}',
        f'{sub}_mergedData_{data_version}_{dType}.P'))

Load windows

In [ ]:
ephys_sources = ['lfp_right', 'lfp_left' ]  #'lfp_right', 'ecog_left', 'ecog_right']
use_stored_windows = True

sub = '108'
json_path = os.path.join(utilsFiles.get_onedrive_path('data'),
                     'featureExtraction_jsons',
                     'ftExtr_spectral_v6.json')
with open(json_path, 'r') as json_data:
    SETTINGS = json.load(json_data)

windows_path = os.path.join(utilsFiles.get_project_path('data', extern_HD=False),
                    'windowed_data_classes_'
                    f'{SETTINGS["WIN_LEN_sec"]}s_'
                    f'{SETTINGS["WIN_OVERLAP_part"]}overlap',
                    SETTINGS['DATA_VERSION'],
                    f'sub-{sub}')

In [ ]:
windows = {}
for dType in ephys_sources[:1]:
    print(f'\tstart {dType}')
    # define path for windows of dType
    dType_fname = (f'sub-{sub}_windows_'
                    f'{SETTINGS["WIN_LEN_sec"]}s_'
                    f'{SETTINGS["DATA_VERSION"]}_{dType}.P')
    dType_win_path = os.path.join(windows_path, dType_fname)

    # check if windows are already available
    if np.logical_and(use_stored_windows,
                      os.path.exists(dType_win_path)):
        print(f'load data from {windows_path}....')
        # windows[dType] = utilsFiles.load_class_pickle(dType_win_path)
        wins = utilsFiles.load_class_pickle(dType_win_path)
        print(f'\tWINDOWS LOADED from {dType_fname} in {windows_path}')

### Explore ssd'd Spectral Features

In [8]:
import lfpecog_features.get_ssd_data as ssd
import lfpecog_analysis.ft_processing_helpers as ftProc
import lfpecog_analysis.get_SSD_timefreqs as ssd_TimeFreq
import lfpecog_plotting.plot_timeFreqs_ssd_psds as plot_ssd_TFs

Plot SSD construction overview

In [ ]:
### PLOT VISUAL SSD OVERVIEW

# # loop over defined frequency bands
# for bw in SETTINGS['SPECTRAL_BANDS']:
#     f_range = SETTINGS['SPECTRAL_BANDS'][bw]
    # check whether to perform SSD
    # (ssd_filt_data,
    #     ssd_pattern,
    #     ssd_eigvals
    # ) = ssd.get_SSD_component(
    #     data_2d=win_dat.T,
    #     fband_interest=f_range,
    #     s_rate=windows.fs,
    #     use_freqBand_filtered=True,
    #     return_comp_n=0,
    # )

#     f, psd = signal.welch(ssd_filt_data, axis=-1,
#                     nperseg=windows.fs, fs=windows.fs)
#     plt.plot(f, psd, label=bw)

        

# plt.xlim(0, 100)
# plt.title(f'WINDOW # {i_w} - {dType.upper()}')
# plt.legend()
# plt.show()

Load SSD timeseries (class)

In [ ]:
# get windowed bands of different dtypes per sub
importlib.reload(ftHelp)
importlib.reload(ssd)
importlib.reload(ssd_TimeFreq)

sub = '022'

# call from feats_extract_multivar.py
ssd_subClass = ssd.get_subject_SSDs(
    sub=sub,
    incl_stn=True,
    incl_ecog=True,
    ft_setting_fname='ftExtr_spectral_v6.json',)




In [ ]:
dat = ssd_subClass.lfp_left.hi_beta
print(dat.shape)
times = np.array(ssd_subClass.lfp_left.times)
print(len(times))

In [ ]:
NAN_ACCEPT = 0.1
c = 0
for i, d in enumerate(dat):
    if any(np.isnan(d)) and (sum(np.isnan(d)) / len(d)) <= NAN_ACCEPT:
        d = dat[i].copy()
        # d[np.isnan(d)] = 0
        # d[~np.isnan(d)] = np.nan
        plt.plot(d + 10*c)
        c += 1

print(f'# of windows with (accepted) NaNs: {c} (out of {len(dat)})')

plt.show()



Explore feature - dyskinesia correlation

In [ ]:
# feature details
WIN_LEN=10
WIN_OVERLAP=0.5
DATA_VERSION='v4.0'
FT_VERSION='v6'
SSD_BROAD=True
INCL_STN_ONLY_PTS=True
IGNORE_PTS=['011', '104', '106']

# analysis details
EPOCH_LEN_SEC = 60

In [ ]:
# find available subs
SUBS = utilsFiles.get_avail_ssd_subs(
    DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION,
    IGNORE_PTS=IGNORE_PTS
)
print(f'SUBS: n={len(SUBS)} ({SUBS})')



In [ ]:
import lfpecog_plotting.plot_descriptive_SSD_PSDs as plot_PSDs

In [ ]:
importlib.reload(ssd_TimeFreq)


TFs = ssd_TimeFreq.get_all_ssd_timeFreqs(
    SUBS=SUBS, FT_VERSION=FT_VERSION,
    DATA_VERSION=DATA_VERSION,
)


In [ ]:
import lfpecog_plotting.prep_and_plot_groupTimeFreqs as groupTFs

In [ ]:
importlib.reload(groupTFs)
source = 'lfp_right'
FIG_TARGET = 'LID'

(ps_arr, arr_times,
 fx, present_subs) = groupTFs.get_group_timefreq(
    source=source, TARGET=FIG_TARGET,
    MIN_SUBS=5, DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION, LOG_POWER=True,)


In [ ]:
importlib.reload(groupTFs)

SHOW_PRES_SUBS = True

FIG_NAME = f'timeFreq_{FIG_TARGET}_{source}'
if SHOW_PRES_SUBS: FIG_NAME += '_subs'

groupTFs.plot_group_timeFreq(
    TARGET=FIG_TARGET, SHOW_PRES_SUBS=SHOW_PRES_SUBS,
    data_arr=ps_arr, freqs=fx, times=arr_times,
    present_subs=present_subs,
    save_figname=FIG_NAME,
    FT_VERSION=FT_VERSION,
    DATA_VERSION=DATA_VERSION,
    source=source,
)

#### Check SSD extraction

In [ ]:
# replicate SSD extraction

importlib.reload(ssd)
# loop over windows

for i_w, win_dat in enumerate(windows.data[:5]):
    win_dat = win_dat.astype(np.float64)    
    # select only rows without missing
    nan_rows = np.array([pd.isna(win_dat[:, i]).any()
                for i in range(win_dat.shape[-1])])
    win_dat = win_dat[:, ~nan_rows]
    win_chnames = list(compress(windows.keys, ~nan_rows))
    win_time = windows.win_starttimes[i_w]
    
    ssds = ssd.SSD_bands_per_window(
        data=win_dat.T, s_rate=windows.fs,
        freq_bands_incl=SETTINGS['SPECTRAL_BANDS'],
    )
 



### Explore Connectivity

In [ ]:
import mne_connectivity as mne_conn

In [ ]:
mne_conn.__version__  # requires newer version for creation of conn results
# additional dependencies: trimesh

In [ ]:
# imported new connectivity functions 
from lfpecog_analysis._connectivity_helpers import (
    load_coordinates,
    process_results,
    plot_results_timefreqs,
    plot_results_patterns,
)

Processing straight from results

In [ ]:
def get_conn_values_sub_side(
    results_df, sub, stn_side, conn_method,
):
    # if conn_method == 'mic':
    conn_data = [
        results_df.iloc[i][conn_method]
        for i in np.arange(results_df.shape[0])
        if (results_df.iloc[i]['subject'] == sub and
            results_df.iloc[i]['seed_target_lateralisation'] == stn_side)
    ][0]

    return conn_data

In [ ]:
def plot_conn_timefreq(
    plot_arr, plot_freqs, plot_times,
    conn_method, title=None,
):

    fig, axis = plt.subplots(1, 1, figsize=(8, 4))

    if conn_method == 'trgc': vmin, vmax = (-1, 1)
    else: vmin, vmax = None, None

    image = axis.imshow(
        plot_arr.T,
        origin="lower",
        extent=(
            plot_times[0] / 60,
            plot_times[-1] / 60,
            plot_freqs[0],
            plot_freqs[-1],
        ),
        vmin=vmin, vmax=vmax,
        aspect="auto",
        cmap="viridis",
    )
    axis.set_xlabel("Time (minutes)")
    axis.set_ylabel("Frequency (Hz)")
    fig.subplots_adjust(right=0.85)
    cbar_axis = fig.add_axes([0.88, 0.15, 0.02, 0.7])
    fig.colorbar(image, cax=cbar_axis, label="Connectivity (A.U.)",)

    # title = f"Method: {METHOD}, sub {sub}, ECoG * {stn_side}-STN"
    if title is not None: axis.set_title(title)

    plt.show()


In [ ]:
SETTINGS = utilsFiles.load_ft_ext_cfg(FT_VERSION='v6')

CONN_FT_PATH = os.path.join(
    utilsFiles.get_project_path('results'),
    'features',
    'connectivity',
    (f'windows_{SETTINGS["WIN_LEN_sec"]}s_'
     f'{SETTINGS["WIN_OVERLAP_part"]}overlap')
)

# take only subjects with ECoG & LFP data
SUBJECTS = [sub for sub in SETTINGS['TOTAL_SUBS']
            if sub.startswith("0")]

METHOD = "mic"

In [ ]:
results, window_times, freqs = process_results(
    method=METHOD, subjects='008', results_path=CONN_FT_PATH
)

In [ ]:
print(results.keys())
print(len(window_times))

In [ ]:
# Plot all time-freqs with Connectivity values
sub = '012'
stn_side = 'ipsilateral'

for sub in SUBJECTS:
    for stn_side in ['contralateral', 'ipsilateral']:

        temp_dat = get_conn_values_sub_side(
            results_df=results, sub=sub,
            stn_side=stn_side, conn_method=METHOD
        )

        # plot_conn_timefreq(plot_arr=temp_dat,
        #            plot_times=window_times,
        #            plot_freqs=freqs,
        #            conn_method=METHOD,
        #            title=f'{METHOD.upper()}: sub {sub}, {stn_side}')


# PM:
# # average over multiple subs and/or sides
# temp_dat = [
#     get_conn_values_sub_side(
#         results_df=results, sub=SUB,
#         stn_side=stn_side, conn_method=METHOD
#     ) for SUB in SUBJECTS[:5]
# ]
# temp_dat = np.nanstd(temp_dat, axis=0)

Plotting via new scripts

In [ ]:
import lfpecog_analysis.process_connectivity as processConn
from statsmodels.regression.mixed_linear_model import MixedLM

In [ ]:
def get_stats_arrays(ipsivalues, ipsi_ids):
    stat_values, stat_ids = {}, {}
    for lid_code, lid_cats in zip(['lid', 'nolid'],
                                    [[1, 2, 3], [0]]):
        # select all arrays LID vs NO-LID, sorted in groups of category
        # [sub1-cat1, sub2-cat1, .., subX-cat1, sub1-cat2, sub2-cat2, ..]
        temp_values = [a for cat in ipsivalues
                        for a in ipsivalues[cat] if cat in lid_cats]
        # get corresponding list of sub-ids
        temp_ids = [s for cat in ipsi_ids for s in ipsi_ids[cat]
                        if cat in lid_cats]  # get sub-ids w/o LID
        temp_ids = [[s] * a.shape[0] for s, a in
                        zip(temp_ids, temp_values)]  # multiply ids with corr array shapes
        temp_ids = np.array([i for l in temp_ids for i in l])  # unpack list of lists    
        # convert into long arrays
        temp_values = np.array([r for a in temp_values for r in a])
        assert temp_values.shape[0] == temp_ids.shape[0], 'incorrect shapes'
        stat_values[lid_code] = temp_values
        stat_ids[lid_code] = temp_ids
    # create single arrays for stats
    stat_values = np.concatenate([stat_values['nolid'],
                                stat_values['lid']])
    stat_labels = np.concatenate([[0] * len(stat_ids['nolid']),
                                [1] * len(stat_ids['lid'])])
    stat_ids = np.concatenate([stat_ids['nolid'],
                            stat_ids['lid']])

    assert (stat_values.shape[0] == stat_labels.shape[0]
            == stat_ids.shape[0]), 'incorrect lengths'

    return stat_values, stat_labels, stat_ids



    
    

In [ ]:
def run_mixEff_wGroups(dep_var, indep_var,
                       groups, TO_ZSCORE=True,
                       ALPHA=.01, RETURN_CI=False,
                       RETURN_GRADIENT=False,):
    """
    # tests sign effect of LID on ephys
    # Model: https://www.statsmodels.org/stable/generated/statsmodels.regression.mixed_linear_model.MixedLM.html
    # Results: https://www.statsmodels.org/stable/generated/statsmodels.regression.mixed_linear_model.MixedLMResults.html
    """

    # z-score ephys values on group level for scaling
    dep_var = (dep_var - np.std(dep_var)) / np.mean(dep_var)
    # define model
    lm_model = MixedLM(
        endog=dep_var,  # dependent variable (ephys score)
        exog=indep_var,  # independent variable (i.e., LID presence, movement)
        groups=groups,  # subjects
        exog_re=None,  # (None)  defaults to a random intercept for each group
    )
    # run and fit model
    lm_results = lm_model.fit()
    # extract results
    fixeff_cf = lm_results._results.fe_params[0]
    pval = lm_results._results.pvalues[0]

    output_list = [fixeff_cf, pval]  # to keep output number dynamic

    if RETURN_CI:
        conf_int = lm_results.conf_int(alpha=ALPHA)[0]
        output_list.append(conf_int)
    
    if RETURN_GRADIENT:
        grad = lm_results._results.params_object()
        output_list.append(grad)

    # return two, three, or four values
    return output_list
    
    # print(f'fixed effect coeff: {fixeff_cf}')  # fixed-effect coeffs
    # print(f'Confidence Interval (alpha: {ALPHA}): {conf_int} (p = {pval.round(5)})')
    # print(lm_results.summary())

In [ ]:
def calc_lmem_freqCoeffs(temp_values,
                         temp_ids,
                         temp_freqs):

    # select freqs 4 - 35 Hz and 60 - 90 Hz
    f_sel = np.logical_or(
        [f >= 4 and f <= 35 for f in temp_freqs],
        [f >= 60 and f <= 90 for f in temp_freqs]
    )
    # correct alpha for multiple comparisons
    ALPHA = .05 / sum(f_sel)
    print(f'corrected ALPHA within LMEM: {round(ALPHA, 5)} rounded')

    # organize long arrays with all values, labels and sub-ids
    (stat_values,
    stat_labels,
    stat_ids) = get_stats_arrays(ipsivalues=temp_values,
                                 ipsi_ids=temp_ids)

    # calculate coeffs and pvalues per frequency bin
    coeffs_freqs, sign_freqs, grads = [], [], []
    for i_f, f in enumerate(temp_freqs):
        # skip irrelevant freqs
        if not f in temp_freqs[f_sel]: continue
        # calculate coeffs for med-effect on values (random slopes for subjects)
        fixEff_cf, pval, grad = run_mixEff_wGroups(
            dep_var=stat_values[:, i_f],
            indep_var=stat_labels,
            groups=stat_ids,
            RETURN_GRADIENT=True,
        )
        sig_bool = pval < ALPHA
        coeffs_freqs.append(fixEff_cf)
        sign_freqs.append(sig_bool)
        grads.append(grad)

    coeffs_freqs = np.array(coeffs_freqs)
    sign_freqs = np.array(sign_freqs)
    grads = np.array(grads)

    return coeffs_freqs, sign_freqs, grads, temp_freqs[f_sel]

In [ ]:
importlib.reload(processConn)
importlib.reload(ssd_TimeFreq)

FT_VERSION='v6'
SETTINGS = utilsFiles.load_ft_ext_cfg(FT_VERSION=FT_VERSION)

# take only subjects with ECoG & LFP data
SUBJECTS = [sub for sub in SETTINGS['TOTAL_SUBS']
            if sub.startswith("0")]
DATA_VERSION = SETTINGS['DATA_VERSION']

# ### get values
# TFs = ssd_TimeFreq.get_all_ssd_timeFreqs(
#     SUBS=SUBJECTS,
#     FT_VERSION=FT_VERSION,
#     DATA_VERSION=DATA_VERSION,
#     # GET_CONNECTIVITY='mic',
#     verbose=False,
# )
print('got tf values')

# TODO: adjust code for PSDs
# connectivity results in TFs [ipsi / contralat]
# PSDs: TFs [lfp_left / lfp_right/ ecog_side]
# merge lfp left and right into pdSeries on index
# take mean -> process similar
# ecog, no averaging needed

### sort and average values into categories
# plot values is dict: contains sides
# side dict: contains different categories 0,1,2,3
plot_values, freqs, value_ids = processConn.get_conn_values_to_plot(
    TFs,
    RETURN_MEAN_per_CAT=False,
    BASELINE_CORRECT=False,
    BASELINE_EXCL_MOVE=True,
    CDRS_SIDE='bilat',
    INCL_CORE_CDRS=True,
    incl_conn_sides=['lfp_left'],
    SELECT_MOVEMENT=False,
    # verbose=self.VERBOSE,
)

In [ ]:
# take data for correct side
side = 'lfp_left'
side_values = plot_values[side]
side_ids = value_ids[side]

coeffs_freqs, sign_freqs, grads, freqlist_coeffs = calc_lmem_freqCoeffs(
    temp_values=side_values,
    temp_ids=side_ids,
    temp_freqs=freqs
)

## Phase explorations

Phase features to add:
- local PAC from De Hempt (ECoG) (beta-phase, gamma-ampl)
    - EEGLAB ASYMM PAC
    - check calculation via entropy of amplitudes per bin
    - or MI-inde
- phase-phase: CHECK CAGNAN BURST WORK
    - connectivity phase differences from Swann et al (phase-coherence) (angle STN versus angle ECoG, compare with imaginary-coherence)

In [9]:
import lfpecog_plotting.phase_plotting as phaseplot

In [ ]:
importlib.reload(phaseplot)

### phase difference
# plt.figure(figsize=(12, 4))
# get both signals
sig1 = ssds.lo_beta.copy()
sig2 = ssd0.lo_beta.copy()
# convert to analytic signal
a1 = signal.hilbert(x=sig1,)
a2 = signal.hilbert(x=sig2,)
# get phase from analytical signal, convert from pi to degree
rad1 = np.angle(a1)
deg1 = np.rad2deg(rad1)
rad2 = np.angle(a2)
deg2 = np.rad2deg(rad2)
# get difference, convert all to positive degrees (-90 -> +270)
rad_diff = rad1 - rad2
deg_diff = deg1 - deg2
mask_deg = deg_diff < 0  # bool-array, 0 for values >= 0
mask_rad = rad_diff < 0
corr_rad = np.array([2 * np.pi] * len(rad_diff)) * mask_rad  # corr array is set 0 for pos diff-values
rad_diff += corr_rad
corr_deg = np.array([360] * len(deg_diff)) * mask_deg  # corr array is set 0 for pos diff-values
deg_diff += corr_deg
plt.plot(rad_diff, label='corr', alpha=.8, ls='dotted')
plt.xlim(0, 5000)
plt.show()

plt.figure(figsize=(12, 4))
plt.plot(sig1)
plt.plot(sig2)
plt.xlim(0, 5000)
plt.show()

phaseplot.plot_rose_axis(radians=rad_diff)

In [ ]:
sig_phase = ssd_014.ecog_right.lo_beta[10].copy()
sig_ampl = ssd_014.ecog_right.broad_gamma[10].copy()
# sig = ssds.lo_beta.copy()

a_phase = signal.hilbert(x=sig_phase,)
a_ampl = signal.hilbert(x=sig_ampl,)
phase = np.angle(a_phase)
phase_deg = np.rad2deg(phase)  #phase * (180 / np.pi)
ampl = abs(a_ampl)
plt.plot(phase)
plt.plot(ampl)
plt.xlim(0, 1000)
plt.yticks([-np.pi, 0, np.pi],
           labels=['-180', '0', '+180'],)
plt.ylabel('Phase (degree)')
plt.xlabel('time (samples)')
plt.show()



### Calculate PAC

In [10]:
from tensorpac import Pac
import lfpecog_features.feats_phase_amp_coupling as fts_pac
import lfpecog_features.extract_ssd_features as ssdFts
from lfpecog_features import bursts_funcs as bursts_funcs


Test low-freq vs Beta/gamma hypothesis

In [11]:

data_path = 'D://Research/CHARITE/projects/dyskinesia_neurophys/data/'
picklepath = os.path.join(
    data_path,
    'windowed_data_classes_10s_0.5overlap',
    'selected_ephys_classes_all'
)

SUB = '105'

picklename = f'ephys_selections_{SUB}.P'

sub_class = load_class_pickle(
    file_to_load=os.path.join(picklepath,
                                picklename),
    convert_float_np64=True
)


print(vars(sub_class.lfp_left_INVOLUNTARY_moveboth_lidlid).keys())
print(sub_class.lfp_left_INVOLUNTARY_moveboth_lidlid.time_arr.shape)
print(sub_class.lfp_left_INVOLUNTARY_moveboth_lidlid.cdrs_arr.shape)

print(sub_class.lfp_left_INVOLUNTARY_moveboth_lidlid.ephys_2d_arr.shape)
print(sub_class.lfp_left_INVOLUNTARY_moveboth_lidlid.band_names)

ERROR while pickle loading of D://Research/CHARITE/projects/dyskinesia_neurophys/data/windowed_data_classes_10s_0.5overlap\selected_ephys_classes_all\ephys_selections_105.P


FileNotFoundError: [Errno 2] No such file or directory: 'D://Research/CHARITE/projects/dyskinesia_neurophys/data/windowed_data_classes_10s_0.5overlap\\selected_ephys_classes_all\\ephys_selections_105.P'

In [ ]:
SETTINGS = utilsFiles.load_ft_ext_cfg(FT_VERSION=='v4')

In [ ]:

pac_values = fts_pac.calculate_PAC_matrix(
    sig_pha=ssd_014.ecog_right.lo_beta,
    sig_amp=ssd_014.ecog_right.narrow_gamma,
    window_times=ssd_014.ecog_right.times,
    fs=ssd_014.ecog_right.fs,
    freq_range_pha=SETTINGS['SPECTRAL_BANDS']['lo_beta'],
    freq_range_amp=SETTINGS['SPECTRAL_BANDS']['narrow_gamma']
)

In [ ]:
# importlib.reload(specFts)

sel1 = 'lfp_left_REST_moveboth_lidno'
sel2 = 'lfp_left_INVOLUNTARY_moveboth_lidlid'

freqs = {'theta': [4, 8], 'beta': [12, 20],
         'gamma': [70, 80]}

# cdrs is only included for debug/analysis here
var_list = ['beta', 'gamma', 'theta', 'cdrs']
var_list = {v: [] for v in var_list}

# get raw SSD-signals
for sel in [sel1, sel2]:
    var_list['theta'].extend(list(getattr(sub_class, sel).ephys_2d_arr[:, 0]))
    var_list['beta'].extend(list(getattr(sub_class, sel).ephys_2d_arr[:, 2]))
    var_list['gamma'].extend(list(getattr(sub_class, sel).ephys_2d_arr[:, -2]))
    var_list['cdrs'].extend(list(getattr(sub_class, sel).cdrs_arr))

# # calculate Theta based on Beta-Gamma
# theta = specFts.get_theta_from_betaGamma(
#     beta_sig=var_list['beta'], fs=2048,
#     gamma_sig=var_list['gamma'],
#     ZSCORE_envs=True,
# )
# var_list['theta_new'] = theta

# calculate smoothed envelops for otger bands
for v in var_list.keys():
    if v in ['cdrs', 'theta_new']: continue
    sig = bursts_funcs.get_envelop(
        np.array(var_list[v]), fs=2048, bandpass_freqs=freqs[v]
    )
    sig = ftHelp.smoothing(sig, fs=2048, win_ms=250,)
    var_list[v] = np.array(sig)


# window resulting arrays

var_windows = {f: [] for f in var_list}

cuts = np.arange(0, len(var_list['theta']), 10*2048)

for i1, i2 in zip(cuts[:-1], cuts[1:]):
    for f in var_windows:
        var_windows[f].append(np.mean(var_list[f][i1:i2]))
        
for f in var_windows: var_windows[f] = np.array(var_windows[f])




In [ ]:
theta_raw = getattr(sub_class, sel).ephys_2d_arr[:, 0]
theta_bp = bursts_funcs.bandpass(theta_raw, freqs=[4, 8], fs=2048,)
theta_analytSig = signal.hilbert(theta_bp, )
theta_env = bursts_funcs.get_envelop(theta_raw, fs=2048,
                                     bandpass_freqs=[4, 8], )
theta_phase = (np.angle(theta_analytSig) + np.pi) / (np.pi * 2)  # normed between 0 and 1


%matplotlib qt
plt.close()

plt.plot(theta_bp, alpha=.5,)
plt.plot(theta_analytSig, alpha=.5,)
plt.plot(theta_phase, alpha=.5,)
# plt.scatter(np.arange(len(theta_raw)), theta_raw, alpha=.7,)
# plt.plot(theta_angle / np.pi)
# plt.plot(np.angle(var_list['theta']))

plt.xlim(0, 30 * 2048)

In [ ]:
plt.close()

mwu_stat, mwu_p = stats.mannwhitneyu(
    var_windows['theta_new'][var_windows['cdrs'] == 0],
    var_windows['theta_new'][var_windows['cdrs'] > 0]
)

print(f'Mann Witney U diff: stat: {mwu_stat},  p: {round(mwu_p, 5)}')

for f in ['theta', 'beta', 'gamma', 'theta_new']:
    res_pearson = stats.pearsonr(var_windows[f], var_windows['cdrs'])

    print(f'\n\t{f.upper()} Pearson R: '
          f'{res_pearson},\n...CI: {res_pearson.confidence_interval()}')
    
    plt.scatter(var_windows['cdrs'], var_windows[f],
                label=f'{f}: R: {round(res_pearson[0], 2)}, '
                f'p = {round(res_pearson[1], 6)}')
plt.legend()
plt.show()

### Develop phase-difference values

In [ ]:
phase_deg = phase * (180 / np.pi)

phase_bins = {}
for bin_start in np.arange(-180, 180, 20):
    bin_sel = np.logical_and(phase_deg > bin_start,
                             phase_deg<(bin_start+20))
    ampl_sel = ampl[bin_sel]

    phase_bins[bin_start] = ampl_sel

In [ ]:
for bin in phase_bins:
    amps = phase_bins[bin]
    plt.hist(amps,alpha=.3)
    ent = stats.entropy(amps)
    plt.title(f'{bin} degree: entropy {ent}')
    plt.close()

#### SSD with meet toolbox (https://github.com/neurophysics/meet)

In [ ]:
import meet.meet as meet

In [ ]:
# get data
i_win = 5
win_dat = data.list_mne_objects[i_win].get_data()  # epochs x channels x times
ch_names = data.list_mne_objects[i_win].ch_names
fs = data.info['sfreq']
nperseg = 1024
bw_ranges = {   'alpha': [8, 12],
                'lo_beta': [12, 20],
                'hi_beta': [20, 35],
                'beta': [12, 35],
                'midgamma': [60, 90]}


In [ ]:
from lfpecog_features import feats_SSD as ssd

In [ ]:
# test and plot SSD functionality

importlib.reload(ssd)

SOURCE_SEL = 'ECOG'
F_BAND_SEL = 'midgamma'
epoch_i = 50
plt.close()
# select 2d data of one source (n-channels x n-samples)
ch_sel = [n.startswith(SOURCE_SEL) for n in ch_names]
epoch_dat = win_dat[epoch_i, ch_sel, :]

fig, axes = plt.subplots(3, 1, figsize=(8,8))

for F_BAND_SEL in ['lo_beta', 'hi_beta', 'beta', 'midgamma']:
    ssd_filt_data, ssd_pattern, ssd_eigvals = ssd.get_SSD_component(
        data_2d=epoch_dat,
        fband_interest=bw_ranges[F_BAND_SEL],
        s_rate=fs,
        use_freqBand_filtered=True,
        return_comp_n=0,
    )
    f, psd = signal.welch(ssd_filt_data, axis=-1, nperseg=fs, fs=fs)

    axes[0].plot(ssd_filt_data, label=F_BAND_SEL)
    axes[1].plot(f, psd, label=F_BAND_SEL)

# psd of origin
for i in range(epoch_dat.shape[0]):
    f, psd = signal.welch(epoch_dat[i, :], nperseg=fs, fs=fs)
    axes[2].plot(f, psd, label=F_BAND_SEL, c='k', alpha=.3,)

axes[0].legend(ncol=4)
axes[1].legend()
axes[0].set_title('SSD filtered bands', fontsize=14, weight='bold',)
axes[0].set_ylabel('LFP (a.u.)', fontsize=14,)
axes[0].set_xlabel('Time (samples, 2048 Hz)', fontsize=14,)
axes[1].set_title('Freq-specific PSD after SSD', fontsize=14, weight='bold',)
axes[2].set_title('Original PSDs of channels', fontsize=14, weight='bold',)

for ax in [1, 2]:
    axes[ax].set_xlim(0, 100)
    axes[ax].set_xlabel('Frequency (Hz)', fontsize=14,)
    axes[ax].set_ylabel('Power (a.u.)', fontsize=14,)

for ax in axes: ax.tick_params(axis='both', labelsize=10)
plt.tight_layout()
figname = 'SSD_example_timeseries_PSD'
# plt.savefig(os.path.join(figpath, 'ft_exploration', 'SSD', figname),
#             dpi=300, facecolor='w',)

plt.close()



In [ ]:
# # plot SSD Components
# for i, b in enumerate(SSD_beta):
#     plt.plot(b, label=f'ch {i}',
#              alpha=(1 - (i * .2)),
#              lw=5 - i)
# # beta1d = SSD_beta.T @ np.atleast_2d(SSD_eigvals).T  # combined signal, not relevant
# # plt.plot(beta1d, label='product', c='k')
# plt.legend()
# plt.show()

In [ ]:
d = pd.read_excel('C://Users/habetsj/Downloads/Mappe1.xlsx', header=1)

d = d[[k for k in d.keys() if k.startswith('UPDRS')]]
nan_sel = [~pd.isna(d.values[i]).any() for i in np.arange(d.shape[0])]
d = d.iloc[nan_sel, :].reset_index(drop=True)
str_sel = [any([isinstance(s, str) for s in d.iloc[i]]) for i in np.arange(d.shape[0])]
d = d.iloc[~np.array(str_sel), :].reset_index(drop=True)

plt.scatter(d.iloc[:, 1], d.iloc[:, 0])
plt.xlabel(d.keys()[1])
plt.ylabel(d.keys()[0])

w = stats.wilcoxon(d.iloc[:, 1], d.iloc[:, 0])
p = stats.pearsonr(d.iloc[:, 1], d.iloc[:, 0])
plt.title(f'{p}\n{w}\n(n = {d.shape[0]} PD patients)')
plt.tight_layout()
plt.savefig('C://Users/habetsj/Downloads/Fallzahl_OFF_ON', dpi=150, facecolor='w')
plt.close()

plt.scatter(d.iloc[:, 1], d.iloc[:, 1] - d.iloc[:, 0])
plt.xlabel(d.keys()[1])
plt.ylabel('UPDRS improvement due to Stimulation')

w = stats.wilcoxon(d.iloc[:, 1], d.iloc[:, 1] - d.iloc[:, 0])
p = stats.pearsonr(d.iloc[:, 1], d.iloc[:, 1] - d.iloc[:, 0])
plt.title(f'{p}\n{w}\n(n = {d.shape[0]} PD patients)')
plt.tight_layout()
plt.savefig('C://Users/habetsj/Downloads/Fallzahl_OFF_change', dpi=150, facecolor='w')
plt.close()